In [4]:
import numpy as np
import cv2
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as trans

import vmdata
import more_trans
import more_sampler

from ezfirstae.loaddata import SlidingWindowBatchSampler

%matplotlib inline

ModuleNotFoundError: No module named 'ezfirstae'

In [3]:
root = vmdata.prepare_dataset_root(9, (8, 0, 0))
normalize = trans.Normalize(*vmdata.get_normalization_stats(root, bw=True))
transform = trans.Compose([trans.ToTensor(), more_trans.RGB2Gray(), normalize])
with vmdata.VideoDataset(root, transform=transform) as vdset:
    sam = more_sampler.SlidingWindowBatchSampler()

AttributeError: module 'more_sampler' has no attribute 'SlidingWindowBatchSampler'